In [26]:
#ライブラリをインポート
import os #OSに依存する様々な機能を利用するためのモジュール(ファイルやディレクトリ操作など)
import re #正規表現を利用するためのモジュール
import csv  #csvファイルを扱うためのモジュール
import math #数学的計算のためのモジュール
import cv2 #画像処理のためのモジュール
import matplotlib.pyplot as plt #グラフ描画のためのモジュール
import numpy as np  #多次元配列計算のためのモジュール
import pandas as pd #データフレームを扱うためのモジュール
from scipy import signal  #信号処理のためのモジュール
from scipy.stats import skew, kurtosis  #歪度と尖度を調べるためのモジュール
from sklearn.model_selection import train_test_split  #データをトレーニング用とテスト用に分けるためのモジュール
from sklearn import preprocessing #データを正規化するためのモジュール
from sklearn.preprocessing import StandardScaler  #データを標準化するためのモジュール
from sklearn.preprocessing import LabelEncoder  #カテゴリ変数を数値化するためのモジュール
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, r2_score #機械学習モデルの性能評価のためのモジュール
import tensorflow as tf #TensorFlow(Googleが開発したオープンソースの機械学習フレームワーク)
from tensorflow import keras  #TensorFlow用のニューラルネットワークライブラリAPI
from tensorflow.keras import layers #ニューラルネットワークのレイヤーを定義するためのモジュール
from keras.utils import np_utils

In [101]:
#メモ
#画像のサイズは縦：横=1088:1920=17:30

In [2]:
#自分のGoogle Driveとドッキング
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#GPUを使うための処理
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
#正解データへのパスとファイル名取得
CollectPath = "/content/drive/MyDrive/DICOMO/正解データ/"
CollectFilename = os.listdir(CollectPath)

In [5]:
#不正解データへのパスとファイル名取得
IncollectPath = "/content/drive/MyDrive/DICOMO/不正解データ/"
IncollectFilename = os.listdir(IncollectPath)

In [6]:
#教師データを格納するリストを定義
img = []

In [7]:
#正解データをグレースケールに変換・縦170横300にリサイズ
for i in CollectFilename:
     image = cv2.imread(CollectPath+i)
     image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
     image = cv2.resize(image, dsize=(300, 170))
     img.append(image)

In [8]:
#不正解データをグレースケールに変換・縦170横300にリサイズ
for i in IncollectFilename:
     image = cv2.imread(IncollectPath+i)
     image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
     image = cv2.resize(image, dsize=(300, 170))
     img.append(image)

In [9]:
#教師ラベルを作成
#渡れる時を1、渡れない時を0とする
CollectLabel = np.ones(len(CollectFilename))
IncollectLabel = np.zeros(len(IncollectFilename))
label = np.concatenate((CollectLabel, IncollectLabel))

In [10]:
#マクロの定義
N = 1 #Encoderの層の数
NUM_HEADS =  16
KEY_DIM = 600
DROPOUT = 0.1 #ドロップアウト率

In [ ]:
img = np.array(img)
img = np_utils.to_categorical(img)

In [12]:
#トレーニングデータとテストデータの分割
x_train, x_test, y_train, y_test = train_test_split(img, label, train_size = 0.8, shuffle = True)

In [15]:
# Define the input shape
input_shape = (170, 300)
output_shape = (1,)

#形を定義(このモジュールは行列でないとダメっぽい)
inputs_encoder = layers.Input(shape=input_shape)

In [16]:
#Encoderに対する入力の形状
x_encoder = layers.Reshape((85, 600))(inputs_encoder)

In [17]:
#Transformer Encoder Layer(BERT)
for i in range(N):
      #Multi-Head-Attention Layer
      attention_encoder = layers.MultiHeadAttention(num_heads=NUM_HEADS, key_dim=KEY_DIM, use_bias=True)(x_encoder, x_encoder, x_encoder)

      #Dropout Layer
      attention_encoder = layers.Dropout(rate=DROPOUT)(attention_encoder)
      #Add & Norm Layer
      attention_encoder = layers.LayerNormalization()(x_encoder + attention_encoder)

      #Feed-Forward-Network
      ffn_encoder = layers.Dense(600 * 4, use_bias=True, activation="relu")(attention_encoder)
      ffn_encoder = layers.Dense(600, use_bias=True)(ffn_encoder)

      #Dropout Layer
      ffn_encoder = layers.Dropout(rate=DROPOUT)(ffn_encoder)
      #Add & Norm Layer
      x_encoder = layers.LayerNormalization()(attention_encoder + ffn_encoder)

In [18]:
x = layers.Flatten()(x_encoder)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

In [19]:
model = keras.Model(inputs=inputs_encoder, outputs=outputs)

In [20]:
# compile model
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
    )

In [21]:
x_train[0].shape

(170, 300)

In [22]:
model.fit(x_train, y_train, batch_size=32, epochs=300, shuffle=True, validation_split=0.2)
y_pred = model.predict(x_test)

Epoch 1/300
13/13 [==============================] - 12s 170ms/step - loss: 15.1760 - accuracy: 0.4651 - val_loss: 1.7424 - val_accuracy: 0.4536
Epoch 2/300
13/13 [==============================] - 2s 140ms/step - loss: 2.8530 - accuracy: 0.5220 - val_loss: 0.8965 - val_accuracy: 0.4536
Epoch 3/300
13/13 [==============================] - 2s 129ms/step - loss: 1.4820 - accuracy: 0.4625 - val_loss: 0.6891 - val_accuracy: 0.5464
Epoch 4/300
13/13 [==============================] - 2s 141ms/step - loss: 1.1597 - accuracy: 0.4470 - val_loss: 0.9643 - val_accuracy: 0.4536
Epoch 5/300
13/13 [==============================] - 2s 140ms/step - loss: 1.0160 - accuracy: 0.4574 - val_loss: 0.6910 - val_accuracy: 0.5464
Epoch 6/300
13/13 [==============================] - 2s 128ms/step - loss: 1.3517 - accuracy: 0.4884 - val_loss: 0.8184 - val_accuracy: 0.4536
Epoch 7/300
13/13 [==============================] - 2s 139ms/step - loss: 1.0730 - accuracy: 0.4858 - val_loss: 0.7113 - val_accuracy: 0.54

In [23]:
accuracy_score(y_pred, y_test)

ValueError: ignored

In [24]:
y_pred

array([[0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.49526778],
       [0.495

In [25]:
y_test

array([1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1.,
       1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1.,
       0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1.,
       0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0.,
       1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1.,
       0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
       0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0.,
       1., 0., 1.])